In [1]:
pip install -r ../../requirements.txt


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import ezkl
import torch
from torch import nn
import json
import os
import time
import scipy
import numpy as np
import matplotlib.pyplot as plt
import statistics
import math

In [3]:
%run -i ../../zkstats/core.py

In [4]:
# init path
os.makedirs(os.path.dirname('shared/'), exist_ok=True)
os.makedirs(os.path.dirname('prover/'), exist_ok=True)
verifier_model_path = os.path.join('shared/verifier.onnx')
prover_model_path = os.path.join('prover/prover.onnx')
verifier_compiled_model_path = os.path.join('shared/verifier.compiled')
prover_compiled_model_path = os.path.join('prover/prover.compiled')
pk_path = os.path.join('shared/test.pk')
vk_path = os.path.join('shared/test.vk')
proof_path = os.path.join('shared/test.pf')
settings_path = os.path.join('shared/settings.json')
srs_path = os.path.join('shared/kzg.srs')
witness_path = os.path.join('prover/witness.json')
# this is private to prover since it contains actual data
sel_data_path = os.path.join('prover/sel_data.json')
# this is just dummy random value
sel_dummy_data_path = os.path.join('shared/sel_dummy_data.json')

=======================  ZK-STATS FLOW =======================

In [5]:
data_path = os.path.join('data.json')
dummy_data_path = os.path.join('shared/dummy_data.json')

data = json.loads(open(data_path, "r").read())['col_name']
data_tensor = torch.reshape(torch.tensor(data),(1,-1, 1))

create_dummy(data_path, dummy_data_path)
dummy_data = json.loads(open(dummy_data_path, "r").read())['col_name']
dummy_data_tensor = torch.reshape(torch.tensor(dummy_data), (1,-1,1))

dummy_theory_output = torch.sqrt(torch.var(dummy_data_tensor, correction = 0))
dummy_data_mean = torch.mean(dummy_data_tensor)
theory_output = torch.sqrt(torch.var(data_tensor, correction = 0))
data_mean = torch.mean(data_tensor)

In [6]:
scales = [2]
selected_columns = ['col_name']
commitment_maps = get_data_commitment_maps(data_path, scales)

In [7]:
# Verifier/ data consumer side:
class verifier_model(nn.Module):
    def __init__(self):
        super(verifier_model, self).__init__()
        self.w = nn.Parameter(data = dummy_theory_output, requires_grad = False)
        self.data_mean = nn.Parameter(data = dummy_data_mean, requires_grad = False)

    def forward(self,X):
        x_mean_cons = torch.abs(torch.sum(X)-X.size()[1]*(self.data_mean))<=torch.abs(0.01*X.size()[1]*self.data_mean)
        # since we square
        return (torch.logical_and(torch.abs(torch.sum((X-self.data_mean)*(X-self.data_mean))-self.w*self.w*X.size()[1])<=torch.abs(0.02*self.w*self.w*X.size()[1]),x_mean_cons ),self.w)


verifier_define_calculation(dummy_data_path, selected_columns,sel_dummy_data_path,verifier_model, verifier_model_path)

/Users/jernkun/Library/Caches/pypoetry/virtualenvs/zkstats-OJpceffF-py3.11/lib/python3.11/site-packages/torch/onnx/symbolic_opset9.py:2174: FutureWarning: 'torch.onnx.symbolic_opset9._cast_Bool' is deprecated in version 2.0 and will be removed in the future. Please Avoid using this function and create a Cast node instead.
  return fn(g, to_cast_func(g, input, False), to_cast_func(g, other, False))


In [8]:
# Prover/ data owner side
print("theory output: ", theory_output)
class prover_model(nn.Module):
    def __init__(self):
        super(prover_model, self).__init__()
        self.w = nn.Parameter(data = theory_output, requires_grad = False)
        self.data_mean = nn.Parameter(data = data_mean, requires_grad = False)
    def forward(self,X):
        x_mean_cons = torch.abs(torch.sum(X)-X.size()[1]*(self.data_mean))<=torch.abs(0.01*X.size()[1]*self.data_mean)
        # since we square
        return (torch.logical_and(torch.abs(torch.sum((X-self.data_mean)*(X-self.data_mean))-self.w*self.w*X.size()[1])<=torch.abs(0.02*self.w*self.w*X.size()[1]),x_mean_cons ),self.w)

prover_gen_settings(data_path, selected_columns, sel_data_path, prover_model,prover_model_path, scales, "resources", settings_path)

theory output:  tensor(14.5557)
==== Generate & Calibrate Setting ====
scale:  [2]
setting:  {"run_args":{"tolerance":{"val":0.0,"scale":1.0},"input_scale":2,"param_scale":2,"scale_rebase_multiplier":10,"lookup_range":[-40670,8],"logrows":16,"num_inner_cols":2,"variables":[["batch_size",1]],"input_visibility":{"Hashed":{"hash_is_public":true,"outlets":[]}},"output_visibility":"Public","param_visibility":"Private"},"num_rows":14432,"total_assignments":1521,"total_const_size":8,"model_instance_shapes":[[1],[1]],"model_output_scales":[0,2],"model_input_scales":[2],"module_sizes":{"kzg":[],"poseidon":[14432,[1]],"elgamal":[0,[0]]},"required_lookups":["Abs",{"GreaterThan":{"a":0.0}}],"check_mode":"UNSAFE","version":"7.0.0","num_blinding_factors":null}


In [9]:
# Here verifier & prover can concurrently call setup since all params are public to get pk.
# Here write as verifier function to emphasize that verifier must calculate its own vk to be sure
verifier_setup(verifier_model_path, verifier_compiled_model_path, settings_path,vk_path, pk_path )

print("=======================================")
# Prover generates proof
print("Theory output: ", theory_output)
prover_gen_proof(prover_model_path, sel_data_path, witness_path, prover_compiled_model_path, settings_path, proof_path, pk_path)

spawning module 0
spawning module 2


==== setting up ezkl ====


spawning module 0
spawning module 2


Time setup: 6.7214109897613525 seconds
Theory output:  tensor(14.5557)
==== Generating Witness ====
witness boolean:  1.0
witness result 1 : 14.5
==== Generating Proof ====


spawning module 0
spawning module 2


proof:  {'instances': [[[12907834141446617622, 16863260785082668294, 2215826187815521673, 2191547160659437828], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [15846044116984004302, 17434658319873045084, 12685703199754313893, 2889686633444970017]]], 'proof': '1d228f3a6465192abe2e43777fc0ae41e0d309c3934336e6233b30d1007b7af307e6521d40dc611e4445cc380f5c695ad1eefe604440deca2c8975c32423127e043315736eb068a11eed3ca64fd57921ebbcd7b2e73c837b76fad7266b70185c0bc7b88431f5c76b34bb88f446aec0e46de76739b1108d059cacca5e4af1fa2f1164b434015826c5dc0fb589643d48720f9e50d960468d469542e4f8a59a3ea030032ebc3160ef9a09b6f880bed1b1530957b3be7ebd0a3872191b03dcac37ca10b12dbbaadb1ed9bf6400aa1cf7020b57a7c4f4cac7912a082f946c49257d871e46abbad2347ddcd3172edd8613d21bc758db434f8a7f0c2876a4205adac6c6251b2bb1985344ed931196da8a2ba0fb4ef25a35a42cfd1cb8e676aaae3295dd08c705f7517185b216cd719c71f8f2aa1b48b2af989a63aad995ec2015f19bab02c453749ce9c5885756b4626bed5c2eee3aab8b2a76769d48d875e370f6

In [10]:
# Verifier verifies
verifier_verify(proof_path, settings_path, vk_path, selected_columns, commitment_maps)

14.5